<a href="https://colab.research.google.com/github/sudarshan-360/Machine-Learning/blob/main/Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"sudarshan360","key":"3434e4053a4cd86de36846e92bb5f33e"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!pip install -q kaggle
!kaggle datasets download -d omkargurav/face-mask-dataset
!unzip -q face-mask-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
face-mask-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
replace data/with_mask/with_mask_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [6]:
import os

path = "/kaggle/input/face-mask-dataset"
print(os.listdir(path))

['data']


In [7]:
print(os.listdir("/kaggle/input/face-mask-dataset/data"))


['without_mask', 'with_mask']


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
tf.__version__

'2.18.0'

In [21]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('data',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 7553 images belonging to 2 classes.


In [22]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('data',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 7553 images belonging to 2 classes.


In [23]:
cnn = tf.keras.models.Sequential()

In [24]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [14]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [25]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [26]:
cnn.add(tf.keras.layers.Flatten())

In [27]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [28]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [29]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [30]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
153/237 ━━━━━━━━━━━━━━━━━━━━ 32s 382ms/step - accuracy: 0.7599 - loss: 0.5229

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


237/237 ━━━━━━━━━━━━━━━━━━━━ 114s 474ms/step - accuracy: 0.7898 - loss: 0.4708 - val_accuracy: 0.8278 - val_loss: 0.4005
Epoch 2/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 127s 536ms/step - accuracy: 0.8860 - loss: 0.2820 - val_accuracy: 0.8985 - val_loss: 0.2484
Epoch 3/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 110s 463ms/step - accuracy: 0.9010 - loss: 0.2396 - val_accuracy: 0.9211 - val_loss: 0.2050
Epoch 4/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 110s 465ms/step - accuracy: 0.9061 - loss: 0.2358 - val_accuracy: 0.9309 - val_loss: 0.1786
Epoch 5/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 127s 536ms/step - accuracy: 0.9248 - loss: 0.1901 - val_accuracy: 0.9419 - val_loss: 0.1534
Epoch 6/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 107s 451ms/step - accuracy: 0.9309 - loss: 0.1777 - val_accuracy: 0.9521 - val_loss: 0.1254
Epoch 7/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 109s 460ms/step - accuracy: 0.9470 - loss: 0.1430 - val_accuracy: 0.9576 - val_loss: 0.1152
Epoch 8/25
237/237 ━━━━━━━━━━━━━━━━━━━━ 114s 480ms/step - accuracy: 0.9461 - loss: 0.14

In [34]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

test_image = image.load_img(img_path, target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

result = cnn.predict(test_image)

if result[0][0] > 0.5:
    prediction = 'Without Masked'
else:
    prediction = 'Masked'

print("Prediction:", prediction)


Saving Screenshot 2025-07-18 at 1.26.03 AM 1.png to Screenshot 2025-07-18 at 1.26.03 AM 1 (2).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Without Masked
